In [9]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as dates

import seaborn as sns
#from pandas_profiling import ProfileReport
from sklearn import metrics

from datetime import datetime
import timeit  
import pickle
from IPython.display import Image

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

pd.options.display.max_rows = 10000
pd.options.display.max_columns = 1000
pd.options.display.max_info_columns = 1000
sns.set_style("whitegrid")
sns.set_context("talk")


In [10]:
from catboost import CatBoostRegressor 
from sklearn.linear_model import LinearRegression

In [11]:
%matplotlib inline

In [13]:
df = pd.read_parquet("df_device_recommendation_input.parquet")
df.dropna(inplace=True)

In [14]:
df.head()

,date_focus,score_DEX_Device_DEX_Device,device_name,user_name,binary_3dx.exe,binary_4l.scr,binary_4lv.scr,binary_acad.exe,binary_acrobat.exe,binary_acrord32.exe,binary_alarmview.exe,binary_alm-client.exe,binary_alteryxgui.exe,binary_atmgr.exe,binary_automate.exe,binary_bdeeasy.exe,binary_bluesattendantconsole.exe,binary_bomgar-acc.exe,binary_brave.exe,binary_china safety month 2020 screensaver.scr,binary_chrome.exe,binary_code.exe,binary_controller.exe,binary_dbeaver.exe,binary_devenv.exe,binary_dsi.dclink.trancollectorpc.exe,binary_dvr.exe,binary_eclipse.exe,binary_excel.exe,binary_fastec.mes.client.exe,binary_firefox.exe,binary_hdbstudio.exe,binary_idea64.exe,binary_iexplore.exe,binary_iispcmi.exe,binary_inspection.exe,binary_inspirescreensaver.scr,binary_java.exe,binary_javaw.exe,binary_jp2launcher.exe,binary_kakaotalk.exe,binary_line.exe,binary_microsoft.skyperoomsystem.exe,binary_mmshell.exe,binary_msaccess.exe,binary_msedge.exe,binary_msedgewebview2.exe,binary_mtb.exe,binary_new_win.scr,binary_notebookclient.exe,binary_notepad++.exe,binary_oms.exe,binary_onenote.exe,binary_other,binary_outlook.exe,binary_pbidesktop.exe,binary_pcsws.exe,binary_play your park campaign.scr,binary_pma.exe,binary_powerpnt.exe,binary_procbook.exe,binary_pulse.exe,binary_putty.exe,binary_pycharm64.exe,binary_rstudio.exe,binary_safety.exe,binary_saplogon.exe,binary_sas.exe,binary_sbus.exe,binary_screensaver_au.scr,binary_securecrt.exe,binary_securitydesk.exe,binary_selfservice.exe,binary_signage.exe,binary_sitekiosk.cefapp.exe,binary_slack.exe,binary_sldworks.exe,binary_snproject.exe,binary_spotfire.dxp.exe,binary_sqldeveloper64w.exe,binary_sqlworkbench64.exe,binary_ssms.exe,binary_station.exe,binary_step v1.0.scr,binary_tableau.exe,binary_teams.exe,binary_toad.exe,binary_tour fans 2.scr,binary_ugraf.exe,binary_viber.exe,binary_visio.exe,binary_vlc.exe,binary_vmware-view.exe,binary_wechat.exe,binary_wemeetapp.exe,binary_wfica32.exe,binary_whatsapp.exe,binary_winproj.exe,binary_winscp.exe,binary_winword.exe,binary_workspaces.exe,binary_wtwindows.exe,binary_zoom.exe,binary_zoomrooms.exe,binary_zrccefclient.exe,model,cpu_model,monitors,total_ram_gb
0,2022-03-28,4.5624,AMC-PF25H95W,thatziyi@eudir.jnj.com,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,4.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,ThinkPad T490,Intel Core i5-8365U CPU @ 1.60GHz,"HP (1920x1080, 23â?³, 3CQ75012VQ-1010101),Wide...",16.0
1,2022-04-04,5.0700,AMC-PF25H95W,thatziyi@eudir.jnj.com,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,10.48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.54,0.0,0.0,0.0,0.0,0.0,ThinkPad T490,Intel Core i5-8365U CPU @ 1.60GHz,"HP (1920x1080, 23â?³, 3CQ75012VQ-1010101),Wide...",16.0
2,2022-04-11,5.8640,AMC-PF25H95W,thatziyi@eudir.jnj.com,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,11.62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,ThinkPad T490,Intel Core i5-8365U CPU @ 1.60GHz,"HP (1920x1080, 23â?³, 3CQ75012VQ-1010101),Wide...",16.0
3,2022-04-18,4.3446,AMC-PF25H95W,thatziyi@eudir.jnj.com,0.0,0.0,0.0,0.0,0.0,1.74,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [15]:
df_train = df[df['date_focus'] < '2022-05-01']
X_train = df_train.copy()
X_train.drop(['date_focus', 'score_DEX_Device_DEX_Device', 'device_name', 'user_name'], axis=1, inplace=True)

y_train = df_train['score_DEX_Device_DEX_Device'].copy()

MemoryError: Unable to allocate 2.61 GiB for an array with shape (103, 3398695) and data type float64

In [7]:
df_test = df[df['date_focus'] >= '2022-05-01']
X_test = df_test.copy()
X_test.drop(['date_focus', 'score_DEX_Device_DEX_Device', 'device_name', 'user_name'], axis=1, inplace=True)

y_test = df_test['score_DEX_Device_DEX_Device'].copy()

In [8]:
# list of cathegorical variables
l_categorical = ['model', 'cpu_model', 'monitors']

### Baseline Model

In [12]:
predictions_train_baseline = np.full(len(y_train), np.mean(y_train))   
predictions_test_baseline = np.full(len(y_test), np.mean(y_train)) 
print('metrics for baseline model:')
print('RMSE - train set:', metrics.mean_squared_error(y_train, predictions_train_baseline, squared=False))
print('RMSE - test set:', metrics.mean_squared_error(y_test, predictions_test_baseline, squared=False)) 
print('R2 - train set:', metrics.r2_score(y_train, predictions_train_baseline))
print('R2 - test set:', metrics.r2_score(y_test, predictions_test_baseline)) 

metrics for baseline model:
RMSE - train set: 1.4795876888692199
RMSE - test set: 1.478824473155947
R2 - trains set: 6.661338147750939e-16
R2 - test set: -0.011510934892384528


### Linear Regression

In [13]:
X_train_no_cat = X_train.drop(l_categorical, axis=1)
X_test_no_cat = X_test.drop(l_categorical, axis=1)
model_lr = LinearRegression()
model_lr.fit(X_train_no_cat, y_train) 
predictions_train_lr = model_lr.predict(X_train_no_cat)
predictions_test_lr = model_lr.predict(X_test_no_cat)
print('metrics for linear regression:')
print('RMSE - train set:', metrics.mean_squared_error(y_train, predictions_train_lr, squared=False))
print('RMSE - test set:', metrics.mean_squared_error(y_test, predictions_test_lr, squared=False)) 
print('R2 - train set:', metrics.r2_score(y_train, predictions_train_lr)) 
print('R2 - test set:', metrics.r2_score(y_test, predictions_test_lr))

metrics for linear regression:
RMSE - train set: 1.3961713607063742
RMSE - test set: 1.406261926616593
R2 - trains set: 0.10957769132112483
R2 - test set: 0.08531878219260736


### Gradient Boosting  (CatBoostRegressor)

In [14]:
start_time = datetime.now()
print('start time:',start_time)
model_gb = CatBoostRegressor(verbose=0, random_seed=710, iterations=1000)  
model_gb.fit(X_train, y_train, cat_features=l_categorical)   
end_time = datetime.now()
print('end time:',end_time)
print('model training time:', end_time - start_time,' [hours:minutes:seconds]')

predictions_train_gb = model_gb.predict(X_train)
predictions_test_gb = model_gb.predict(X_test)

print('metrics for gradient boosting:')
print('RMSE - train set:', metrics.mean_squared_error(y_train, predictions_train_gb, squared=False))
print('RMSE - test set:', metrics.mean_squared_error(y_test, predictions_test_gb, squared=False)) 
print('R2 - train set:', metrics.r2_score(y_train, predictions_train_gb)) 
print('R2 - test set:', metrics.r2_score(y_test, predictions_test_gb))

start time: 2022-06-09 12:57:59.540816
end time: 2022-06-09 13:04:54.284859
model training time: 0:06:54.744043  [hours:minutes:seconds]
metrics for gradient boosting:
RMSE - train set: 1.1577137825223327
RMSE - test set: 1.2062145460024774
R2 - trains set: 0.3877610026922109
R2 - test set: 0.3270443454918861


In [45]:
### Saving model as pickle 
pickle.dump(model_gb, open('model_gb_ver0.pickle', 'wb'))

### Using the model

In [46]:
### Loading model
model = pickle.load(open('model_gb_ver0.pickle', 'rb'))

In [47]:
x = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 110.47,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.71, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        'ThinkCentre M920q', 'Intel Core i5-9600T CPU @ 2.30GHz',
        'SyncMaster (3840x2160, 74.5â?³, H1AK500000-E00),FLARE (1280x800, 10.2â?³)',
        8.0]

In [51]:
# the variables in array x have the following meaning:
list(X_train.columns)

['binary_3dx.exe',
 'binary_4l.scr',
 'binary_4lv.scr',
 'binary_acad.exe',
 'binary_acrobat.exe',
 'binary_acrord32.exe',
 'binary_alarmview.exe',
 'binary_alm-client.exe',
 'binary_alteryxgui.exe',
 'binary_atmgr.exe',
 'binary_automate.exe',
 'binary_bdeeasy.exe',
 'binary_bluesattendantconsole.exe',
 'binary_bomgar-acc.exe',
 'binary_brave.exe',
 'binary_china safety month 2020 screensaver.scr',
 'binary_chrome.exe',
 'binary_code.exe',
 'binary_controller.exe',
 'binary_dbeaver.exe',
 'binary_devenv.exe',
 'binary_dsi.dclink.trancollectorpc.exe',
 'binary_dvr.exe',
 'binary_eclipse.exe',
 'binary_excel.exe',
 'binary_fastec.mes.client.exe',
 'binary_firefox.exe',
 'binary_hdbstudio.exe',
 'binary_idea64.exe',
 'binary_iexplore.exe',
 'binary_iispcmi.exe',
 'binary_inspection.exe',
 'binary_inspirescreensaver.scr',
 'binary_java.exe',
 'binary_javaw.exe',
 'binary_jp2launcher.exe',
 'binary_kakaotalk.exe',
 'binary_line.exe',
 'binary_microsoft.skyperoomsystem.exe',
 'binary_mmshel

In [48]:
model.predict(x)

9.530942177172621